In [30]:
import pandas as pd
import numpy as np

from joblib import dump, load
from zipfile import ZipFile, ZIP_DEFLATED
import glob
import itertools
from uuid import uuid4
import json
from os import path, stat
from sys import getsizeof
from datetime import datetime
from time import time

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize

from xgboost import XGBClassifier

# Функции

In [20]:
if not path.isfile('models.json'):
  f = open('models.json', 'w')
  f.write('{"data": {}, "models": {}}')
  f.close()

def save_model(model, x, y, description={}):
  id = str(uuid4())
  dump(model, f'{id}.joblib')

  f =  open('models.json', 'r')
  j = json.load(f)
  f.close()

  t = time()
  model.predict(x)
  t = time() - t

  j['models'][id] =  {
      'model_type': str(type(model)),
      'score': {
          'f1': f1_score(y, model.predict(x)),
          'recall': recall_score(y, model.predict(x)),
          'precision': precision_score(y, model.predict(x)),
          'accuracy': accuracy_score(y, model.predict(x))
      },
      'params': model.get_params(),
      'description': dict(description,
                          date=str(datetime.now()),
                          predict_time=t,
                          memory_size=getsizeof(model),
                          file_size=stat(f'{id}.joblib').st_size)}
  f = open('models.json', 'w')
  json.dump(j, f, ensure_ascii=False, indent=4)
  f.close()


def grid_search(model, params, x_train, y_train, x_test, y_test,
                f1_min=0.9, recall_min=0.9, precision_min=0.9, description={}):
  keys = list(params.keys())
  for param in itertools.product(*[params[j] for j in keys]):
    par = {v: param[i] for i, v in enumerate(keys)}
    model.set_params(**par)
    t = time()
    model.fit(x_train, y_train)
    t = time() - t

    f = f1_score(y_test, model.predict(x_test))
    r = recall_score(y_test, model.predict(x_test))
    p = precision_score(y_test, model.predict(x_test))

    if f >= f1_min and r >= recall_min and p >= precision_min:
      print('\nTest f1 score:', f)
      print('Test recall score:', r)
      print('Test precision score:', p)
      print('\nTrain f1 score:', f1_score(y_train, model.predict(x_train)))
      print('Train recall score:', recall_score(y_train, model.predict(x_train)))
      print('Train precision score:', precision_score(y_train, model.predict(x_train)))

      # save_model(model, x_test, y_test, description=dict(description, time=t))

# Подготовка

In [21]:
data = pd.ExcelFile('Кардио1.xlsx')
data.sheet_names
data = data.parse('Суперфинал (3 регистра)', index_col='История болезни')
data

,Пол,Возраст,Мочевина (1-е сутки),Креатинин (1-е сутки),СКФ (CKD-EPI 2021),АСТ (1-е сутки),АЛТ (1-е сутки),CRP,Глюкоза (1-е сутки),Лейкоциты (1-е сутки),...,Лимфоциты (1-е сутки),Нейтрофильно-лимфоцитарное соотношение,Выжил?,Степень тяжести,"D-димер, ед.",АГ,СД,ИБС (с ХСН),ХОБЛ + астма,ХБП (3-5)
История болезни,,,,,,,,,,,,,,,,,,,,,
103,жен,60,13.7,106.0,63.396226,42.00,47.00,7.31,4.7,16.40,...,0.90,16.333333,да,тяжелая,250.0,да,нет,нет,нет,нет
116,жен,80,25.0,350.0,11.000000,41.00,20.00,107.00,11.9,25.80,...,0.10,244.000000,нет,тяжелая,NaN,да,нет,нет,нет,нет
127,жен,64,15.3,103.0,52.000000,31.04,36.23,6.00,9.3,14.40,...,2.10,5.476190,да,тяжелая,2310.0,да,да,нет,нет,нет
136,жен,61,4.7,83.0,69.000000,40.00,14.00,244.00,13.1,10.00,...,1.20,7.166667,нет,тяжелая,1100.0,да,да,нет,нет,нет
147,муж,74,12.2,141.0,45.000000,24.85,30.14,8.04,11.9,14.50,...,2.30,4.956522,да,тяжелая,620.0,да,да,да,нет,нет
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851394,жен,66,6.6,98.0,55.000000,20.00,12.90,25.00,5.5,10.30,...,2.20,3.136364,да,легкая,6254.0,да,да,да,нет,да
851832,муж,60,5.6,65.0,104.000000,22.00,50.00,15.30,5.9,12.21,...,1.25,7.384000,да,средняя,2018.0,да,нет,нет,нет,нет
852893,жен,86,16.2,154.0,28.000000,15.00,16.00,204.00,6.2,7.60,...,1.64,3.390244,нет,тяжелая,4054.0,да,нет,да,нет,да


In [22]:
data.isnull().sum()

Пол                                        0
Возраст                                    0
Мочевина (1-е сутки)                       0
Креатинин (1-е сутки)                      0
СКФ (CKD-EPI 2021)                         0
АСТ (1-е сутки)                            3
АЛТ (1-е сутки)                            3
CRP                                        3
Глюкоза (1-е сутки)                        1
Лейкоциты (1-е сутки)                      0
Тромбоциты (1-е сутки                      0
Нейтрофилы (1-е сутки)                     5
Лимфоциты (1-е сутки)                      5
Нейтрофильно-лимфоцитарное соотношение     5
Выжил?                                     0
Степень тяжести                            0
D-димер, ед.                              20
АГ                                         0
СД                                         0
ИБС (с ХСН)                                0
ХОБЛ + астма                               0
ХБП (3-5)                                  0
dtype: int

In [23]:
data = data.fillna(data.mean())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


In [24]:
data.isnull().sum()

Пол                                       0
Возраст                                   0
Мочевина (1-е сутки)                      0
Креатинин (1-е сутки)                     0
СКФ (CKD-EPI 2021)                        0
АСТ (1-е сутки)                           0
АЛТ (1-е сутки)                           0
CRP                                       0
Глюкоза (1-е сутки)                       0
Лейкоциты (1-е сутки)                     0
Тромбоциты (1-е сутки                     0
Нейтрофилы (1-е сутки)                    0
Лимфоциты (1-е сутки)                     0
Нейтрофильно-лимфоцитарное соотношение    0
Выжил?                                    0
Степень тяжести                           0
D-димер, ед.                              0
АГ                                        0
СД                                        0
ИБС (с ХСН)                               0
ХОБЛ + астма                              0
ХБП (3-5)                                 0
dtype: int64

In [25]:
data.loc[data.Пол == 'муж', 'Пол'] = 0.
data.loc[data.Пол == 'жен', 'Пол'] = 1.
data.loc[data['Выжил?'] == 'нет', 'Выжил?'] = 0.
data.loc[data['Выжил?'] == 'да', 'Выжил?'] = 1.
data.loc[data['АГ'] == 'нет', 'АГ'] = 0.
data.loc[data['АГ'] == 'да', 'АГ'] = 1.
data.loc[data['СД'] == 'нет', 'СД'] = 0.
data.loc[data['СД'] == 'да', 'СД'] = 1.
data.loc[data['ИБС (с ХСН)'] == 'нет', 'ИБС (с ХСН)'] = 0.
data.loc[data['ИБС (с ХСН)'] == 'да', 'ИБС (с ХСН)'] = 1.
data.loc[data['ХОБЛ + астма'] == 'нет', 'ХОБЛ + астма'] = 0.
data.loc[data['ХОБЛ + астма'] == 'да', 'ХОБЛ + астма'] = 1.
data.loc[data['ХБП (3-5)'] == 'нет', 'ХБП (3-5)'] = 0.
data.loc[data['ХБП (3-5)'] == 'да', 'ХБП (3-5)'] = 1.
data.loc[data['Степень тяжести'] == 'тяжелая', 'Степень тяжести'] = 0.
data.loc[data['Степень тяжести'] == 'тяжелый', 'Степень тяжести'] = 0.
data.loc[data['Степень тяжести'] == 'легкая', 'Степень тяжести'] = 1.
data.loc[data['Степень тяжести'] == 'средняя', 'Степень тяжести'] = 0.5

In [26]:
st_t = pd.get_dummies(data[['Степень тяжести']])
data = data.drop('Степень тяжести', axis=1)
data = data.join(st_t)

In [27]:
data = data.astype('float64')

In [28]:
data

,Пол,Возраст,Мочевина (1-е сутки),Креатинин (1-е сутки),СКФ (CKD-EPI 2021),АСТ (1-е сутки),АЛТ (1-е сутки),CRP,Глюкоза (1-е сутки),Лейкоциты (1-е сутки),...,Выжил?,"D-димер, ед.",АГ,СД,ИБС (с ХСН),ХОБЛ + астма,ХБП (3-5),Степень тяжести_0.0,Степень тяжести_0.5,Степень тяжести_1.0
История болезни,,,,,,,,,,,,,,,,,,,,,
103,1.0,60.0,13.7,106.0,63.396226,42.00,47.00,7.31,4.7,16.40,...,1.0,250.000000,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
116,1.0,80.0,25.0,350.0,11.000000,41.00,20.00,107.00,11.9,25.80,...,0.0,4595.896061,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
127,1.0,64.0,15.3,103.0,52.000000,31.04,36.23,6.00,9.3,14.40,...,1.0,2310.000000,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
136,1.0,61.0,4.7,83.0,69.000000,40.00,14.00,244.00,13.1,10.00,...,0.0,1100.000000,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
147,0.0,74.0,12.2,141.0,45.000000,24.85,30.14,8.04,11.9,14.50,...,1.0,620.000000,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851394,1.0,66.0,6.6,98.0,55.000000,20.00,12.90,25.00,5.5,10.30,...,1.0,6254.000000,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
851832,0.0,60.0,5.6,65.0,104.000000,22.00,50.00,15.30,5.9,12.21,...,1.0,2018.000000,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
852893,1.0,86.0,16.2,154.0,28.000000,15.00,16.00,204.00,6.2,7.60,...,0.0,4054.000000,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0


In [29]:
x_train, x_test, y_train, y_test = train_test_split(data.drop('Выжил?', axis=1), data['Выжил?'], random_state=5, test_size=0.1)

# KNeighbors

In [ ]:
model = KNeighborsClassifier()
grid = GridSearchCV(model, {'n_neighbors': range(1, 20), 'p': range(1, 10)})

grid.fit(x_train, y_train)

print('\tKNeighbors')
print('\ngrid.best_params_:', grid.best_params_)

model.set_params(**grid.best_params_)
model.fit(x_train, y_train)

f = f1_score(y_test, model.predict(x_test))
r = recall_score(y_test, model.predict(x_test))
p = precision_score(y_test, model.predict(x_test))

print('\nTest f1 score:', f)
print('Test recall score:', r)
print('Test precision score:', p)
print('\nTrain f1 score:', f1_score(y_train, model.predict(x_train)))
print('Train recall score:', recall_score(y_train, model.predict(x_train)))
print('Train precision score:', precision_score(y_train, model.predict(x_train)))

if f > 0.85 and r > 0.85 and p > 0.85:
  print('\nSave model')
  save_model(model, x_test, y_test, description={'model': 'KNeighborsClassifier'})

	KNeighbors

grid.best_params_: {'n_neighbors': 6, 'p': 1}

Test f1 score: 0.5
Test recall score: 0.3888888888888889
Test precision score: 0.7

Train f1 score: 0.7449664429530202
Train recall score: 0.6981132075471698
Train precision score: 0.7985611510791367


# SVC

In [ ]:
model = SVC()
grid = GridSearchCV(model, {'C': range(1, 5), 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']})

grid.fit(x_train, y_train)

print('\tSVC')
print('\ngrid.best_params_:', grid.best_params_)

model.set_params(**grid.best_params_)
model.fit(x_train, y_train)

f = f1_score(y_test, model.predict(x_test))
r = recall_score(y_test, model.predict(x_test))
p = precision_score(y_test, model.predict(x_test))

print('\nTest f1 score:', f)
print('Test recall score:', r)
print('Test precision score:', p)
print('\nTrain f1 score:', f1_score(y_train, model.predict(x_train)))
print('Train recall score:', recall_score(y_train, model.predict(x_train)))
print('Train precision score:', precision_score(y_train, model.predict(x_train)))

if f > 0.85 and r > 0.85 and p > 0.85:
  print('\nSave model')
  save_model(model, x_test, y_test, description={'model': 'SVC'})

grid.best_params_: {'C': 2, 'kernel': 'linear'}
Train score: 0.7491289198606271
Test score: 0.59375


# DecisionTreeClassifier

In [ ]:
print('\nDecisionTreeClassifier')

for i in range(5):
  print('\nI: ', i)
  model = DecisionTreeClassifier()
  grid_search(model, {'criterion': ['gini', 'entropy'], 'max_depth': range(1, 20)},
              x_train, y_train, x_test, y_test,
              f1_min=0.9, recall_min=0.9, precision_min=0.0,
              description={'model': 'DecisionTreeClassifier'})


DecisionTreeClassifier

I:  0

I:  1

I:  2

I:  3

I:  4


# RandomForestClassifier

In [ ]:
print('\nRandomForestClassifier')

for i in range(2):
  print('\nI: ', i)
  model = RandomForestClassifier(random_state=int(time()))
  grid_search(model, {'n_estimators': range(1, 21), 'max_depth': range(1, 100)},
              x_train, y_train, x_test, y_test,
              f1_min=0.85, recall_min=0.85, precision_min=0.85, 
              description={'model': 'RandomForestClassifier'})


RandomForestClassifier

I:  0

Test f1 score: 0.8888888888888888
Test recall score: 0.8888888888888888
Test precision score: 0.8888888888888888

Train f1 score: 0.8940397350993377
Train recall score: 0.8490566037735849
Train precision score: 0.9440559440559441

Test f1 score: 0.8888888888888888
Test recall score: 0.8888888888888888
Test precision score: 0.8888888888888888

Train f1 score: 0.8976897689768977
Train recall score: 0.8553459119496856
Train precision score: 0.9444444444444444

I:  1


# XGBoost

In [54]:
print('\nXGBClassifier')

model = XGBClassifier(random_state=1)
grid_search(model, {'n_estimators': range(1, 21), 'max_depth': range(1, 100)},
            x_train, y_train, x_test, y_test,
            f1_min=0.85, recall_min=0.85, precision_min=0.85, 
            description={'model': 'XGBClassifier'})


XGBClassifier


# Сохранение моделей

In [ ]:
with ZipFile('models.zip', 'w') as z:
  filenames = glob.glob('*.joblib')
  for i in filenames:
    z.write(i, i, ZIP_DEFLATED)
  filenames = glob.glob('*.json')
  for i in filenames:
    z.write(i, i, ZIP_DEFLATED)